In [ ]:
import random
import warnings
import regex as re
import ast

In [ ]:
warnings.filterwarnings('ignore')

## Step 1: Generate init -> goal pairs ##

In [ ]:
n=5

In [ ]:
def get_blocks(n):
  bag_of_blocks=[]
  for i in range(n):
    bag_of_blocks.append(i+1)
    
  return bag_of_blocks
print(get_blocks(n))

In [ ]:
from itertools import combinations
from itertools import permutations

def get_partitions(blocks):
    if not blocks:
        return [[]]
    
    partitions = []
    for i in range(1, len(blocks) + 1):
        for subset in combinations(blocks, i):
            remaining = [b for b in blocks if b not in subset]
            for partition in get_partitions(remaining):
                new_partition = [list(subset)] + list(partition)
                sorted_partition = tuple(sorted(tuple(sorted(group)) for group in new_partition))
                if sorted_partition not in partitions:
                    partitions.append(sorted_partition)
                    
    return partitions

blocks = get_blocks(n)
all_config=set()
all_configs = set(get_partitions(blocks))

print(all_configs)


In [ ]:
def get_all_permed(all_configs):
    w_permuations=[]
    for config in all_configs:
        #print(config)
        #config=((1,3),(2,))
        config_as_list=list(config)
        for idx,tower in enumerate(config_as_list):
                # print(idx,tower)
                if(len(tower)>1):
                    #print(idx,tower)
                    permutations_of_tower=tuple(permutations(tower))
                    modified_configs=[]

                    for element in permutations_of_tower:
                    
                        modified_config=config_as_list.copy()
                        modified_config[idx]=element
                        #äprint(f'Modified permutation:{modified_config}')
                        modified_configs.append(tuple(modified_config))
                    
                    for con in modified_configs:
                        w_permuations.append(tuple(con))
                    #print(perm_towers)
    return w_permuations
            
print(get_all_permed(all_configs))


In [ ]:
def get_all_states(n):

    blocks=get_blocks(n)
    all_configs = set(get_partitions(blocks))
    permutations=get_all_permed(all_configs)

    for element in permutations:
        if  element not in all_configs:
            all_configs.add(element)
    
    return all_configs

    

In [ ]:
all_states=get_all_states(n)
len(all_states)

In [ ]:
print(all_states)

In [ ]:
colours=['Red','Blue','Green','Yellow','Pink',
        'Orange','Purple','Brown','White','Violet',
        'Indigo','Ochre','Black','Magenta','Crimson',
        'Turquoise','Periwinkle','Grey','Emerald']

In [ ]:
#trial-run
selected_colours=random.sample(colours,n)
selected_colours

In [ ]:
#making init -> goal state pairs
import pandas as pd
def make_init_goal_lists(all_states):
    init=[]
    goal=[]
    list_all_states=list(all_states)
    for idx1,state_1 in enumerate(list_all_states):
        #print(idx1)
        for idx2, state_2 in enumerate(list_all_states):
            if idx1 != idx2:
                #print(f'works! {idx1}, {idx2}')
                init.append(list_all_states[idx1])
                goal.append(list_all_states[idx2])
    return init,goal

#trial-run
init,goal=make_init_goal_lists(all_states)                
for i in range(len(init)):
    print(f'For {n} blocks: Initial {init[i]} -> Goal {goal[i]}')


In [ ]:
d={'init':init,'goal':goal}
problems=pd.DataFrame(data=d)

In [ ]:
problems

In [ ]:
#shuffling problems to avoid clumps of same config across dataset splits
problems=problems.sample(frac=1).reset_index(drop=True)
problems

In [ ]:
train=problems[100000:162000].copy()
test=problems[162000:179000].copy()
val=problems[179000:].copy()
print(f"Size of splits: \nTrain: {len(train['init'])} \nValidation: {len(val['init'])}\nTest: {len(test['init'])}\n\nTOTAL:{len(val['init'])+len(test['init'])+len(train['init'])}")

In [ ]:
train.reset_index(drop=True,inplace=True)
train

In [ ]:
val.reset_index(drop=True,inplace=True)
val

In [ ]:
test.reset_index(drop=True,inplace=True)
test

In [ ]:
def fill_colour(state,colour_map):
    #debug line
    #print(f"Type received: {type(state)} - Value: {state}")
    # ((1,), (2,))
    return tuple(
        tuple(colour_map.get(block, block) for block in tower)
        for tower in state
    )

In [ ]:
fixcolour_set1=['red','blue','green','yellow','pink','grey']
fixcolour_set2=['orange','purple','brown','white','violet','teal']

In [ ]:
#varcolour_set1=random.sample(fixcolour_set1,n)
#varcolour_set2=random.sample(fixcolour_set2,n)
#print(f'Set 1:{varcolour_set1} \nSet 2: {varcolour_set2}')

In [ ]:
#colours=[varcolour_set1,varcolour_set2]
colours

In [ ]:
def get_col_map(colours):

    keys=colours[0]+colours[1]
    values=colours[1]+colours[0]
    colour_map=dict(zip(keys,values))
    return colour_map

In [ ]:
colour_map=get_col_map(colours)
colour_map

In [ ]:
#colours each row by randomly selecting one of two colour sets for each row
def get_row_colored(df,i):
    selected_colours={}
    idx_set=random.sample([0,1],1)[0]
    values=colours[idx_set]
    keys=[i for i in range(1,n+1)]
    selected_colours=dict(zip(keys,values))
    print(f'Selected colours for this round: {selected_colours}')
    
    df['init'][i]=fill_colour(df['init'][i],selected_colours)
    df['goal'][i]=fill_colour(df['goal'][i],selected_colours)

In [ ]:
#colours the entire dataframe row-by-row
def color_me_pretty(df):
    for i in range(len(df['init'])):
        print('-----------------------------------------------------------------\n\n')
        print(f"Getting row {i} coloured, {df['init'][i]} -> {df['goal'][i]}\n")
        get_row_colored(df,i=i)
        print(f"\nAfter colouring row {i} , {df['init'][i]} -> {df['goal'][i]}\n")
    

In [ ]:
color_me_pretty(train)

In [ ]:
train

In [ ]:
#train.to_csv(f'../data/train_{n}_blocks')

In [ ]:
#train.to_csv(f'../data/train_{n}_blocks_contn')

In [ ]:
color_me_pretty(test)

In [ ]:
test

In [ ]:
#test.to_csv(f'../data/test_{n}_blocks')

In [ ]:
test.to_csv(f'../data/test_{n}_blocks_contn')

In [ ]:
color_me_pretty(val)

In [ ]:
val

In [ ]:
#val.to_csv(f'../data/val_{n}_blocks')

In [ ]:
val.to_csv(f'../data/val_{n}_blocks_contn')

In [ ]:
test

In [ ]:
#contaminated test set
contaminated_test=test[:3793].copy()
contaminated_test

In [ ]:
def swap_colours(state,colour_map):
    return tuple(
        tuple(colour_map[block] if block in colour_map else block for block in tower) for tower in state)

In [ ]:
def contiminate(df,colour_map):
    for i in range(len(df['init'])):
        
        print('-----------------------------------------------------------------\n\n')
        print(f'Colour map is: {colour_map}\n\n')
        print(f"Swapping colours of row {i}:, {df['init'][i]} -> {df['goal'][i]}\n")
        
        #swap colour for each row
        df['init'][i]=swap_colours(df['init'][i],colour_map)
        df['goal'][i]=swap_colours(df['goal'][i],colour_map)

        print(f"\nAfter swapping colours of row {i} , {df['init'][i]} -> {df['goal'][i]}\n")


In [ ]:
colour_map=get_col_map(colours)
contiminate(contaminated_test,colour_map)

In [ ]:
contaminated_test

In [ ]:
#contaminated_test.to_csv(f'../data/contaminated_test_{n}_blocks')

## Step 2: Generation of prompt akin to planbench dataset ##

In [ ]:
def create_ic(state):
    state = ast.literal_eval(state)
    init=""
    clear=[]
    print(f'State in create_ic {state}' )
    for tower in state:
        tower_l=list(tower)
        # print(f'Tower: {tower_l}')

        for i,block in enumerate(tower_l):
            
            if len(tower_l) > 1:
                # print(f'Height of tower : {len(tower_l)}')
                # print(f'Index of block: {i}')

                #lowermost block in the tower
                if i == 0:
                    # print(f' the {block} block is on the table,')
                    init=init+f' the {str(block).lower()} block is on the table,'

                #highest block in the tower
                elif i == len(tower_l)-1:
                    # print(f' the {block} block is on top of the {tower_l[i-1]} block,')
                    init=init+f' the {str(block).lower()} block is on top of the {str(tower_l[i-1]).lower()} block,'
                    # print(f' the {block} block is clear,')
                    clear.append({str(block).lower()})

                #everything else
                else:
                    # print(f' the {block} block is on top of the {tower_l[i-1]} block,')
                    init=init+f' the {str(block).lower()} block is on top of the {str(tower_l[i-1]).lower()} block,'


            else:
                # print(f'Height of tower : {len(tower)}')
                # print(f' the {str(block)} block is on the table,')
                init=init+f' the {str(block).lower()} block is on the table,'
                clear.append({str(block).lower()})

    #print('#######',init)

    clear_str=""
    for element in clear:
        for block in element:
            clear_str=f'the {block.lower()} block is clear, '+clear_str

    init=clear_str+' the hand is empty,'+init
    init=init[:-1] #remove the tail ,

    return init

In [ ]:
#i_state=(('Purple', 'Violet', 'Teal', 'Orange', 'Brown'),)
#i_state= (('Teal', 'White'), ('Purple',))
i_state=(('Grey',), ('Pink',), ('Blue', 'Yellow'), ('Green',))
init=create_ic(i_state)
print(init)

In [ ]:
ic=init.split(',')

In [ ]:
ic

In [ ]:
def create_goal(state):
    state = ast.literal_eval(state)
    goal=""
    all_clear=True
    print(f'State in create_goal {state}' )
    for tower in state:
        tower_l=list(tower)
        #print(tower_l)
    
        for i,block in enumerate(tower_l):
            
            if len(tower_l) > 1:

                all_clear=False #found a tower with multiple blocks

                if i>0:
                    #print(f' the {str(block)} block is on top of the {str(tower_l[i-1])} block,')
                    goal=goal+f' the {str(block).lower()} block is on top of the {str(tower_l[i-1]).lower()} block,'

    #for a config where NO block is on top of another one
    if all_clear:
        for tower in state:
            tower_l=list(tower)
            for block in tower_l:
                    goal=goal+f' the {str(block).lower()} block is clear,'
    
    return goal[:-1]

In [ ]:
g_state=(('Blue', 'Green', 'Grey'), ('Pink',), ('Yellow',))
#g_state=(('White',), ('Teal',), ('Purple',))
goal=create_goal(g_state)
goal

In [ ]:
gs=goal.split(',')

In [ ]:
gs

In [ ]:
import unifiedplanning_blocksworld as ubs
import planbench as pb
from importlib import reload

In [ ]:
reload(pb)
reload(ubs)

In [ ]:
problem=ubs.BlocksworldProblem()
#init,goal=pb.parse_problem(pbstring)
pb.parse_planbench_initial_condition(problem, init)
pb.parse_planbench_goal_state(problem, goal)
print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')

In [ ]:
plan=problem.generate_plan()

In [ ]:
planstring=str(plan.plan)
plan_strings=planstring.split('\n')
plan_strings=[plan_string.strip() for plan_string in plan_strings]
plan_strings=plan_strings[1:]

In [ ]:
plan_strings

In [ ]:
def form_action_string(action):

    if 'unstack' in action:
        m=re.search(r"unstack\((\w+), (\w+)\)",action)
        return f"unstack the {m.group(1)} block from on top of the {m.group(2)} block"
    
    elif 'pick_up' in action:
        m=re.search(r"pick_up\((\w+)",action)
        return f"pick up the {m.group(1)} block"
    
    elif 'put_down' in action:
        m=re.search(r"put_down\((\w+)",action)
        return f"put down the {m.group(1)} block"
    
    elif 'stack' in action:
        m=re.search(r"stack\((\w+), (\w+)\)",action)
        return f"stack the {m.group(1)} block on top of the {m.group(2)} block"

In [ ]:
plan_strings = [form_action_string(plan_string) for plan_string in plan_strings] 
plan_strings = ['[PLAN]'] + plan_strings + ['[PLAN END]']

In [ ]:
plan_strings

In [ ]:
#load train data
n=3
import pandas as pd
trainData=pd.read_csv(f'../data/train_{n}_blocks')
trainData.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
trainData

In [ ]:
#remove demonstration rows from training data
demo_rows=11
rows_to_drop=list(range(0,demo_rows))
trainData=trainData.drop(rows_to_drop, axis=0).reset_index(drop=True)
trainData

In [ ]:
#extract demonstrations
demo_rows=15
demo=trainData[:demo_rows].copy()
demo.reset_index(drop=True,inplace=True)
demo

In [ ]:
valData=pd.read_csv(f'../data/val_{n}_blocks')
valData.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
valData

In [ ]:
#remove demonstration rows from training data
# rows_to_drop=list(range(0,demo_rows))
# trainData=trainData.drop(rows_to_drop, axis=0).reset_index(drop=True)
# trainData

In [ ]:
def create_ubs_object(demo_row):
    problem=ubs.BlocksworldProblem()
    init=create_ic(demo_row['init'])
    goal=create_goal(demo_row['goal'])

    print(f'parsed initial conditions: {init}')
    print(f'parsed initial conditions: {goal}')

    pb.parse_planbench_initial_condition(problem, init)
    pb.parse_planbench_goal_state(problem, goal)

    print(f'\n\nBlocksworld Problem Initial Values:{problem.initial_values}')
    print(f'\nBlocksworld Problem Goal Condition:{problem.goals}')
    plan=problem.generate_plan()
    return init,goal,plan
    

In [ ]:
def add_plans_to_demo(demo):
    
    demo['plan']=''

    for index, row in demo.iterrows():
        print('Demo row:',index, row[0],'->',row[1])
        init,goal,plan=create_ubs_object(row)

        demo.at[index, 'init']=init
        demo.at[index, 'goal']=goal

        #string from plan object
        plan_strings=str(plan.plan).strip().split('\n')
        plan_strings=plan_strings[1:]
        plan_strings = [form_action_string(plan_string) for plan_string in plan_strings] 
        plan_strings = ['[PLAN]'] + plan_strings + ['[PLAN END]']

        print(f'------PLAN STRINGS------ AFTER PROCESSING: {plan_strings}')
        processed_plan='\n'.join(plan_strings)
        demo.at[index, 'plan']=processed_plan
        
    

In [ ]:
add_plans_to_demo(demo)

In [ ]:
demo

In [ ]:
valData['demo_init']=''
valData['demo_goal']=''
valData['demo_plan']=''

In [ ]:

for i in range(len(valData['init'])):

    # Sample a random row from the demo dataframe
    random_demo_row = demo.sample(n=1).iloc[0]

    # Define the index of the row in 'train' where you want to add the demo data
    val_index = i 

    # Assign values from demo to the corresponding columns in train
    valData.loc[val_index, ['demo_init', 'demo_goal', 'demo_plan']] = random_demo_row[['init', 'goal', 'plan']].values


In [ ]:
valData

In [ ]:
trainData['init'][51]

In [ ]:
trainData['goal'][51]

In [ ]:
trainData['demo_init'][51]

In [ ]:
trainData['demo_goal'][51]

In [ ]:
print(trainData['demo_plan'][51])

In [ ]:
valData.to_csv(f'../data/Oneshot_val_{n}_blocks')